<a href="https://colab.research.google.com/github/kurniacf/BigData_LSH/blob/main/LSH_5025201073_Kurnia_Cahya_Febryanto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignments 3
- Name: Kurnia Cahya Febryanto
- Student ID: 5025201073
- Class: Big Data A
- Lecturer: Abdul Munif, S.Kom., M.Sc.

## Source:
1. https://www.uber.com/en-ID/blog/lsh/
2. https://stackoverflow.com/questions/56816537/cant-find-kaggle-json-file-in-google-colab
3. https://spark.apache.org/docs/latest/api/python/index.html
4. https://spark.apache.org/docs/latest/ml-features.html#locality-sensitive-hashing

# Initialization

## Checking the Environment

In [ ]:
!java --version
!python --version

openjdk 11.0.18 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
Python 3.9.16


## Installing Apache Spark (PySpark)

In [ ]:
## Installing Apache Spark (PySpark)

!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 21.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=e578ae2f87e6fc6e0dac813e2b9e036a0f85b340c6c53f76d3c7f3b576a0f83e
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark


## Initialize Apache Spark context

In [ ]:
# Import Apache Spark SQL
from pyspark.sql import SparkSession

# Create Spark Session/Context
# We are using local machine with all the CPU cores [*]
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Hello Pyspark") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [ ]:
# Check spark session
print(spark)

# Data Mining Task

The LSH task always consists of three steps:

1. Converting original data into vectors
2. Calculate the hash using MinHash algorithm
3. Searching the similar pairs using k-Nearest Neighbor, or join algorithm.

## Downloading the dataset

In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# PLEASE USE YOUR OWN KEY
# Download your own key according to this instruction https://github.com/Kaggle/kaggle-api#api-credentials

import json
api_token = {"username": "kurniacahyaf",
             "key": "ec42e5adad1902e810597fb501b71190"}

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
# Download from https://www.kaggle.com/datasets/urbanbricks/wikipedia-promotional-articles

!kaggle datasets download -d urbanbricks/wikipedia-promotional-articles

 91% 182M/201M [00:01<00:00, 108MB/s]
100% 201M/201M [00:01<00:00, 117MB/s]


## Extract Dataset

In [ ]:
!unzip wikipedia-promotional-articles.zip

Archive:  wikipedia-promotional-articles.zip
  inflating: good.csv                
  inflating: promotional.csv         


## check files in current directory

In [ ]:
!ls -la

total 783144
drwxr-xr-x 1 root root      4096 Mar  8 22:18 .
drwxr-xr-x 1 root root      4096 Mar  8 21:55 ..
drwxr-xr-x 4 root root      4096 Mar  7 18:12 .config
-rw-r--r-- 1 root root 475685227 Oct 27  2019 good.csv
-rw-r--r-- 1 root root 115360355 Oct 27  2019 promotional.csv
drwxr-xr-x 1 root root      4096 Mar  7 18:14 sample_data
-rw-r--r-- 1 root root 210863294 Mar  8 22:08 wikipedia-promotional-articles.zip


## Read the dataset

In [ ]:
# Read CSV (promotional.csv)
df = spark.read.option("header", True).csv("/content/promotional.csv")
df.printSchema()

root
 |-- text: string (nullable = true)
 |-- advert: string (nullable = true)
 |-- coi: string (nullable = true)
 |-- fanpov: string (nullable = true)
 |-- pr: string (nullable = true)
 |-- resume: string (nullable = true)
 |-- url: string (nullable = true)



In [ ]:
# Add an ID for the dataset
from pyspark.sql.functions import monotonically_increasing_id

newsDF = df.withColumn("id", monotonically_increasing_id())
newsDF.show()

+--------------------+------+---+------+---+------+--------------------+---+
|                text|advert|coi|fanpov| pr|resume|                 url| id|
+--------------------+------+---+------+---+------+--------------------+---+
|1 Litre no Namida...|     0|  0|     1|  0|     0|https://en.wikipe...|  0|
|1DayLater was fre...|     1|  1|     0|  0|     0|https://en.wikipe...|  1|
|1E is a privately...|     1|  0|     0|  0|     0|https://en.wikipe...|  2|
|1Malaysia pronoun...|     1|  0|     0|  0|     0|https://en.wikipe...|  3|
|The Jerusalem Bie...|     1|  0|     0|  0|     0|https://en.wikipe...|  4|
|1st Round Enterpr...|     0|  0|     0|  1|     0|https://en.wikipe...|  5|
|2ergo is a provid...|     1|  0|     0|  0|     0|https://en.wikipe...|  6|
|2N Telekomunikace...|     1|  0|     0|  0|     0|https://en.wikipe...|  7|
|A 3D printing mar...|     1|  0|     0|  0|     0|https://en.wikipe...|  8|
|3DR is an America...|     1|  1|     0|  0|     0|https://en.wikipe...|  9|

In [ ]:
# Get the totals row
newsDF.count()

23837

## 1. Prepare the tokenizer
We transform the input into tokenized words

In [ ]:
# Prepare the tokenizer
from pyspark.ml.feature import Tokenizer

# create a tokenizer object to tokenize the text
tokenizer = Tokenizer(inputCol="text", outputCol="words")
# tokenize the text in the dataframe
wordsDF = tokenizer.transform(newsDF)

# show the resulting dataframe
wordsDF.show()

+--------------------+------+---+------+---+------+--------------------+---+--------------------+
|                text|advert|coi|fanpov| pr|resume|                 url| id|               words|
+--------------------+------+---+------+---+------+--------------------+---+--------------------+
|1 Litre no Namida...|     0|  0|     1|  0|     0|https://en.wikipe...|  0|[1, litre, no, na...|
|1DayLater was fre...|     1|  1|     0|  0|     0|https://en.wikipe...|  1|[1daylater, was, ...|
|1E is a privately...|     1|  0|     0|  0|     0|https://en.wikipe...|  2|[1e, is, a, priva...|
|1Malaysia pronoun...|     1|  0|     0|  0|     0|https://en.wikipe...|  3|alaysia, prono...|
|The Jerusalem Bie...|     1|  0|     0|  0|     0|https://en.wikipe...|  4|[the, jerusalem, ...|
|1st Round Enterpr...|     0|  0|     0|  1|     0|https://en.wikipe...|  5|[1st, round, ente...|
|2ergo is a provid...|     1|  0|     0|  0|     0|https://en.wikipe...|  6|[2ergo, is, a, pr...|
|2N Telekomunikace...| 

In [ ]:
# Vectorize the dataset
from pyspark.ml.feature import CountVectorizer

# define the size of the vocabulary and the minimum document frequency
vocabSize=1000

# create a CountVectorizer object and fit it on the tokenized data
cvModel = CountVectorizer(inputCol="words", outputCol="features", vocabSize=vocabSize, minDF=10).fit(wordsDF)

# transform the tokenized data into a vectorized format
vectorizedDF = cvModel.transform(wordsDF)

# show the resulting dataframe
vectorizedDF.show()

+--------------------+------+---+------+---+------+--------------------+---+--------------------+--------------------+
|                text|advert|coi|fanpov| pr|resume|                 url| id|               words|            features|
+--------------------+------+---+------+---+------+--------------------+---+--------------------+--------------------+
|1 Litre no Namida...|     0|  0|     1|  0|     0|https://en.wikipe...|  0|[1, litre, no, na...|(1000,[0,1,2,3,4,...|
|1DayLater was fre...|     1|  1|     0|  0|     0|https://en.wikipe...|  1|[1daylater, was, ...|(1000,[0,1,2,3,4,...|
|1E is a privately...|     1|  0|     0|  0|     0|https://en.wikipe...|  2|[1e, is, a, priva...|(1000,[0,1,2,3,4,...|
|1Malaysia pronoun...|     1|  0|     0|  0|     0|https://en.wikipe...|  3|alaysia, prono...|(1000,[0,1,2,3,4,...|
|The Jerusalem Bie...|     1|  0|     0|  0|     0|https://en.wikipe...|  4|[the, jerusalem, ...|(1000,[0,1,2,3,4,...|
|1st Round Enterpr...|     0|  0|     0|  1|     0|

## 2. Fit/train an LSH Model

In [ ]:
from pyspark.ml.feature import MinHashLSH

# Define the MinHashLSH model with the desired input and output columns, and number of hash tables
mh = MinHashLSH(inputCol="features", outputCol="hashValues", numHashTables=3)

# Train the model using the vectorized data
LSHmodel = mh.fit(vectorizedDF)

# Apply the trained LSH model to the vectorized data and show the results
LSHmodel.transform(vectorizedDF).show()


+--------------------+------+---+------+---+------+--------------------+---+--------------------+--------------------+--------------------+
|                text|advert|coi|fanpov| pr|resume|                 url| id|               words|            features|          hashValues|
+--------------------+------+---+------+---+------+--------------------+---+--------------------+--------------------+--------------------+
|1 Litre no Namida...|     0|  0|     1|  0|     0|https://en.wikipe...|  0|[1, litre, no, na...|(1000,[0,1,2,3,4,...|[[7051277.0], [28...|
|1DayLater was fre...|     1|  1|     0|  0|     0|https://en.wikipe...|  1|[1daylater, was, ...|(1000,[0,1,2,3,4,...|[[4.0937066E7], [...|
|1E is a privately...|     1|  0|     0|  0|     0|https://en.wikipe...|  2|[1e, is, a, priva...|(1000,[0,1,2,3,4,...|[[1.940785E7], [2...|
|1Malaysia pronoun...|     1|  0|     0|  0|     0|https://en.wikipe...|  3|alaysia, prono...|(1000,[0,1,2,3,4,...|[[7051277.0], [28...|
|The Jerusalem Bie...| 

## 3. Searching the similar pairs/items for a key "united" "states"

In [ ]:
# Get the index of the word "united" and "states" in the vocabulary
print(cvModel.vocabulary.index("united"))
print(cvModel.vocabulary.index("states"))

92
198


In [ ]:
# Convert the input with 2 words into a 1000-size vector
# If the words exist in the index, we will give the value 1.0, otherwise 0.0
# Final result: key = [0, 0, ..., 1.0, ..., 1.0, 0, ..., 0]
from pyspark.ml.linalg import Vectors
key = Vectors.sparse(vocabSize, {cvModel.vocabulary.index("united"): 1.0, cvModel.vocabulary.index("states"): 1.0})

In [ ]:
# Define the number of neighbors
k = 40

In [ ]:
# Search inside the LSH model that we already trained
resultDF = LSHmodel.approxNearestNeighbors(vectorizedDF, key, k)
resultDF.show()

+--------------------+------+---+------+---+------+--------------------+----------+--------------------+--------------------+--------------------+------------------+
|                text|advert|coi|fanpov| pr|resume|                 url|        id|               words|            features|          hashValues|           distCol|
+--------------------+------+---+------+---+------+--------------------+----------+--------------------+--------------------+--------------------+------------------+
|Phyllis C. Borzi ...|     0|  0|     0|  0|     1|https://en.wikipe...|      2593|[phyllis, c., bor...|(1000,[0,2,3,7,8,...|[[4.0937066E7], [...|               0.8|
|Metech Recycling ...|     1|  0|     0|  0|     0|https://en.wikipe...|8589935638|[metech, recyclin...|(1000,[0,1,2,3,5,...|[[4.0937066E7], [...|0.8823529411764706|
|TangoDown is an A...|     1|  0|     0|  0|     0|https://en.wikipe...|8589943000|[tangodown, is, a...|(1000,[0,1,2,6,7,...|[[4.0937066E7], [...|0.8823529411764706|
|Cal

In [ ]:
# Save the result into CSV
import pandas as pd
data = resultDF.toPandas()
data.to_csv("result.csv")

## Check Result.csv

In [ ]:
import pandas as pd

# read the CSV file into a Pandas dataframe
df = pd.read_csv('result.csv')

# display the first 5 rows of the dataframe
print(df.head())

   Unnamed: 0                                               text  advert  coi  \
0           0  Phyllis C. Borzi was the Obama administration'...       0    0   
1           1  Metech Recycling is a United States based elec...       1    0   
2           2  TangoDown is an American manufacturer and mark...       1    0   
3           3  Calvary Christian High School is a private, co...       1    0   
4           4  Marine Corps Installations West MCI WEST is th...       1    0   

   fanpov  pr  resume                                                url  \
0       0   0       1      https://en.wikipedia.org/wiki/Phyllis%20Borzi   
1       0   0       0  https://en.wikipedia.org/wiki/Metech%20Incorpo...   
2       0   0       0            https://en.wikipedia.org/wiki/TangoDown   
3       0   0       0  https://en.wikipedia.org/wiki/Calvary%20Christ...   
4       0   0       0  https://en.wikipedia.org/wiki/Marine%20Corps%2...   

           id                                           